In [1]:
# Import libraries
import pandas as pd

# Importing preprocessing tools from sklearn:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# 1) mean_monthly_rent (Base Dataset) Analysis

In [2]:
# load the mean_monthly_rent dataste
mean_monthly_rent = pd.read_csv('./2017_lloguer_preu_trim.csv')

In [3]:
mean_monthly_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Any             584 non-null    int64  
 1   Trimestre       584 non-null    int64  
 2   Codi_Districte  584 non-null    int64  
 3   Nom_Districte   584 non-null    object 
 4   Codi_Barri      584 non-null    int64  
 5   Nom_Barri       584 non-null    object 
 6   Lloguer_mitja   584 non-null    object 
 7   Preu            546 non-null    float64
dtypes: float64(1), int64(4), object(3)
memory usage: 36.6+ KB


In [4]:
mean_monthly_rent.isna().sum()

Any                0
Trimestre          0
Codi_Districte     0
Nom_Districte      0
Codi_Barri         0
Nom_Barri          0
Lloguer_mitja      0
Preu              38
dtype: int64

In [5]:
mean_monthly_rent.duplicated().value_counts()

False    584
Name: count, dtype: int64

In [6]:
# Data Transformation

# Splitting column ('Lloguer_mitja') by its classes `Lloguer mitjà mensual (Euros/mes)` and `Lloguer mitjà per superfície
# (Euros/m2 mes)` to new columns with their corresponding  ('Preu') values.
mean_monthly_rent_pivoted = mean_monthly_rent.pivot_table(
    index=['Trimestre', 'Codi_Districte','Nom_Districte', 'Codi_Barri', 'Nom_Barri'], 
    columns='Lloguer_mitja', 
    values='Preu', 
    aggfunc='first'
)

# Flattening the DataFrame's multi-level index to make it easier to analyze and work with.
mean_monthly_rent_pivoted.reset_index(inplace=True)

# Removing the label for the DataFrame's columns level for a cleaner table format.
mean_monthly_rent_pivoted.columns.name = None

# Sorting the DataFrame by district name, neighborhood name, and quarter to organize the data sequentially.
mean_monthly_rent_pivoted = mean_monthly_rent_pivoted.sort_values(
    by=['Nom_Districte', 'Nom_Barri', 'Trimestre']
)

# Outputting the pivot table to check the reorganized rent data.
mean_monthly_rent_pivoted

,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer mitjà mensual (Euros/mes),Lloguer mitjà per superfície (Euros/m2 mes)
3,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,15.01
71,2,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",930.13,15.99
140,3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",927.16,16.16
208,4,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",914.78,15.66
1,1,1,Ciutat Vella,2,el Barri Gòtic,905.26,14.14
...,...,...,...,...,...,...,...
230,4,5,Sarrià-Sant Gervasi,27,el Putxet i el Farró,938.20,15.32
22,1,5,Sarrià-Sant Gervasi,24,les Tres Torres,1516.52,16.12
90,2,5,Sarrià-Sant Gervasi,24,les Tres Torres,1646.76,16.76
159,3,5,Sarrià-Sant Gervasi,24,les Tres Torres,1757.00,16.07


In [7]:
mean_monthly_rent_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 273 entries, 3 to 227
Data columns (total 7 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Trimestre                                    273 non-null    int64  
 1   Codi_Districte                               273 non-null    int64  
 2   Nom_Districte                                273 non-null    object 
 3   Codi_Barri                                   273 non-null    int64  
 4   Nom_Barri                                    273 non-null    object 
 5   Lloguer mitjà mensual (Euros/mes)            273 non-null    float64
 6   Lloguer mitjà per superfície (Euros/m2 mes)  273 non-null    float64
dtypes: float64(2), int64(3), object(2)
memory usage: 17.1+ KB


In [8]:
mean_monthly_rent_pivoted.describe(include='float64').transpose()

,count,mean,std,min,25%,50%,75%,max
Lloguer mitjà mensual (Euros/mes),273.0,829.380403,253.210128,302.52,680.8,771.78,912.51,1882.42
Lloguer mitjà per superfície (Euros/m2 mes),273.0,12.638059,2.479090,4.45,11.0,12.64,13.97,20.61


In [9]:
mean_monthly_rent_pivoted.describe(include='object')

,Nom_Districte,Nom_Barri
count,273,273
unique,10,70
top,Nou Barris,"Sant Pere, Santa Caterina i la Ribera"
freq,45,4


# 2) city_noise Analysis

In [10]:
# Load city_noise dataste
city_noise = pd.read_csv('./2017_poblacio_exposada_barris_mapa_estrategic_soroll_bcn_long.csv')
city_noise.head()

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Concepte,Rang_soroll,Valor
0,1,Ciutat Vella,1,el Raval,TOTAL_D,<40 dB,7.73%
1,1,Ciutat Vella,1,el Raval,TOTAL_D,40-45 dB,26.98%
2,1,Ciutat Vella,1,el Raval,TOTAL_D,45-50 dB,7.38%
3,1,Ciutat Vella,1,el Raval,TOTAL_D,50-55 dB,11.97%
4,1,Ciutat Vella,1,el Raval,TOTAL_D,55-60 dB,19.85%


In [11]:
city_noise.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18980 entries, 0 to 18979
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Codi_Districte  18980 non-null  int64 
 1   Nom_Districte   18980 non-null  object
 2   Codi_Barri      18980 non-null  int64 
 3   Nom_Barri       18980 non-null  object
 4   Concepte        18980 non-null  object
 5   Rang_soroll     18980 non-null  object
 6   Valor           18980 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.0+ MB


In [12]:
city_noise.isna().sum()

Codi_Districte    0
Nom_Districte     0
Codi_Barri        0
Nom_Barri         0
Concepte          0
Rang_soroll       0
Valor             0
dtype: int64

In [13]:
city_noise.describe(include='object')

,Nom_Districte,Nom_Barri,Concepte,Rang_soroll,Valor
count,18980,18980,18980,18980,18980
unique,10,73,26,10,3104
top,Nou Barris,el Raval,TOTAL_D,<40 dB,0.00%
freq,3380,260,730,1898,9953


In [14]:
# Data Transformation

# Splitting 'Rang_soroll' values under different noise level ranges as separate columns.
city_noise_pivoted = city_noise.pivot_table(
    index=['Codi_Districte','Nom_Districte', 'Codi_Barri', 'Nom_Barri', 'Concepte'], 
    columns='Rang_soroll', 
    values='Valor', 
    aggfunc='first'
)

# Flattening the DataFrame's multi-level index into a single level to simplify the table structure.
city_noise_pivoted.reset_index(inplace=True)

# Removing the name label for the DataFrame column levels for a cleaner look.
city_noise_pivoted.columns.name = None

# Converting percentage strings in the DataFrame to float values, for columns that contain percentages.
for column in city_noise_pivoted.columns:
    # Check for a percentage symbol in the first row of the column.
    if '%' in str(city_noise_pivoted[column].iloc[0]):  
        city_noise_pivoted[column] = city_noise_pivoted[column].str.rstrip('%').astype('float') / 100.0

# Sorting the DataFrame for better readability and organization, by neighborhood name, district name, and noise concept.
city_noise_pivoted = city_noise_pivoted.sort_values(
    by=['Nom_Barri', 'Nom_Districte', 'Concepte']
)

# Displaying the DataFrame
city_noise_pivoted

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Concepte,40-45 dB,45-50 dB,50-55 dB,55-60 dB,60-65 dB,65-70 dB,70-75 dB,75-80 dB,<40 dB,>=80 dB
1482,9,Sant Andreu,58,Baró de Viver,FFCC_D,0.0792,0.0386,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.8822,0.0
1483,9,Sant Andreu,58,Baró de Viver,FFCC_DEN,0.0768,0.0500,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.8732,0.0
1484,9,Sant Andreu,58,Baró de Viver,FFCC_E,0.0796,0.0236,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.8968,0.0
1485,9,Sant Andreu,58,Baró de Viver,FFCC_N,0.0051,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9949,0.0
1486,9,Sant Andreu,58,Baró de Viver,GI_TR_D,0.2190,0.3734,0.1438,0.1268,0.1067,0.0303,0.0000,0.0000,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,5,Sarrià-Sant Gervasi,24,les Tres Torres,TRANSIT_DEN,0.1235,0.1932,0.1589,0.1861,0.1516,0.0961,0.0824,0.0072,0.0010,0.0
620,5,Sarrià-Sant Gervasi,24,les Tres Torres,TRANSIT_E,0.2104,0.1717,0.1712,0.1788,0.1091,0.1005,0.0185,0.0000,0.0398,0.0
621,5,Sarrià-Sant Gervasi,24,les Tres Torres,TRANSIT_N,0.1715,0.1699,0.1767,0.1078,0.1000,0.0164,0.0000,0.0000,0.2577,0.0
622,5,Sarrià-Sant Gervasi,24,les Tres Torres,VIANANTS_D,0.0090,0.0028,0.0003,0.0000,0.0000,0.0000,0.0000,0.0000,0.9879,0.0


In [15]:
city_noise_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1898 entries, 1482 to 623
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Codi_Districte  1898 non-null   int64  
 1   Nom_Districte   1898 non-null   object 
 2   Codi_Barri      1898 non-null   int64  
 3   Nom_Barri       1898 non-null   object 
 4   Concepte        1898 non-null   object 
 5   40-45 dB        1898 non-null   float64
 6   45-50 dB        1898 non-null   float64
 7   50-55 dB        1898 non-null   float64
 8   55-60 dB        1898 non-null   float64
 9   60-65 dB        1898 non-null   float64
 10  65-70 dB        1898 non-null   float64
 11  70-75 dB        1898 non-null   float64
 12  75-80 dB        1898 non-null   float64
 13  <40 dB          1898 non-null   float64
 14  >=80 dB         1898 non-null   float64
dtypes: float64(10), int64(2), object(3)
memory usage: 237.2+ KB


In [16]:
city_noise_pivoted.to_csv('city_noise_pivoted.csv', index=False)

# 3) Data Merge

In [17]:
# Data Merging

# Merging two datasets on district code 'Codi_Districte' and neighborhood code 'Codi_Barri'.
# We use a left join so that all rows from the first dataset (mean_monthly_rent_pivoted) are retained,
# and only the matching rows from the second dataset (city_noise_pivoted) based on the join keys.
merged_data = pd.merge(mean_monthly_rent_pivoted, city_noise_pivoted, 
                       on=['Codi_Districte', 'Codi_Barri'], 
                       how='left', 
                       suffixes=('_rent', '_noise'))

# Removing columns that are duplicated due to the merge ('Nom_Districte_noise', 'Nom_Barri_noise').
merged_data = merged_data.drop(['Nom_Districte_noise', 'Nom_Barri_noise'], axis=1)

# Sorting the merged data by district name, neighborhood name, and quarter to organize the data.
merged_data = merged_data.sort_values(by=['Nom_Districte_rent', 'Nom_Barri_rent','Trimestre'])

# Displaying the first few rows
merged_data.head()

,Trimestre,Codi_Districte,Nom_Districte_rent,Codi_Barri,Nom_Barri_rent,Lloguer mitjà mensual (Euros/mes),Lloguer mitjà per superfície (Euros/m2 mes),Concepte,40-45 dB,45-50 dB,50-55 dB,55-60 dB,60-65 dB,65-70 dB,70-75 dB,75-80 dB,<40 dB,>=80 dB
0,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,15.01,FFCC_D,0.0007,0.0010,0.0001,0.0026,0.0000,0.0000,0.0000,0.0,0.9956,0.0
1,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,15.01,FFCC_DEN,0.0005,0.0010,0.0003,0.0002,0.0024,0.0000,0.0000,0.0,0.9956,0.0
2,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,15.01,FFCC_E,0.0006,0.0009,0.0002,0.0026,0.0000,0.0000,0.0000,0.0,0.9957,0.0
3,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,15.01,FFCC_N,0.0008,0.0001,0.0026,0.0000,0.0000,0.0000,0.0000,0.0,0.9965,0.0
4,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,15.01,GI_TR_D,0.1715,0.0424,0.0163,0.0187,0.0157,0.0132,0.0122,0.0,0.7100,0.0


In [18]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7098 entries, 0 to 7097
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Trimestre                                    7098 non-null   int64  
 1   Codi_Districte                               7098 non-null   int64  
 2   Nom_Districte_rent                           7098 non-null   object 
 3   Codi_Barri                                   7098 non-null   int64  
 4   Nom_Barri_rent                               7098 non-null   object 
 5   Lloguer mitjà mensual (Euros/mes)            7098 non-null   float64
 6   Lloguer mitjà per superfície (Euros/m2 mes)  7098 non-null   float64
 7   Concepte                                     7098 non-null   object 
 8   40-45 dB                                     7098 non-null   float64
 9   45-50 dB                                     7098 non-null   float64
 10  

In [19]:
merged_data.isna().sum()

Trimestre                                      0
Codi_Districte                                 0
Nom_Districte_rent                             0
Codi_Barri                                     0
Nom_Barri_rent                                 0
Lloguer mitjà mensual (Euros/mes)              0
Lloguer mitjà per superfície (Euros/m2 mes)    0
Concepte                                       0
40-45 dB                                       0
45-50 dB                                       0
50-55 dB                                       0
55-60 dB                                       0
60-65 dB                                       0
65-70 dB                                       0
70-75 dB                                       0
75-80 dB                                       0
<40 dB                                         0
>=80 dB                                        0
dtype: int64

In [20]:
merged_data.duplicated().value_counts()

False    7098
Name: count, dtype: int64

# PCA

In [21]:
# Data Preprocessing

# Extracting only the columns with numerical data types ('float64', 'int64') for PCA analysis.
numerical_cols = merged_data.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Defining the names of columns that contain percentage values stored as strings.
percent_cols = ['40-45 dB', '45-50 dB', '50-55 dB', '55-60 dB', '60-65 dB', 
                '65-70 dB', '70-75 dB', '75-80 dB', '<40 dB', '>=80 dB']

# Creating a list of features for PCA which includes percentage columns and two columns related to rental prices.
features = percent_cols + ['Lloguer mitjà mensual (Euros/mes)', 'Lloguer mitjà per superfície (Euros/m2 mes)']

# Selecting the data for PCA using the features list.
X = merged_data[features]

# Initializing the StandardScaler to normalize the features.
scaler = StandardScaler()

# Initializing PCA with 2 components to reduce the dimensionality for an initial visualization.
pca = PCA(n_components=2)

# Setting up a pipeline to scale the data and then apply PCA.
pipeline = Pipeline([('scaler', scaler), ('pca', pca)])

# Applying the pipeline to the features data to perform scaling and PCA.
X_pca = pipeline.fit_transform(X)

# Displaying the first five rows of the transformed features after PCA.
print(f' Five first rows of X_pca are \n {X_pca[:5]}') 
print('-------------------------------------------------')
# Printing the ratio of variance explained by the first two principal components.
print(f'ratio of variance explained by the first two principal components')
print(f' PC1         PC2 \n {pca.explained_variance_ratio_}') 

 Five first rows of X_pca are 
 [[-1.94454168 -0.89663031]
 [-1.94345799 -0.8989612 ]
 [-1.94494623 -0.89646897]
 [-1.94984259 -0.89527799]
 [-0.51744155 -0.8147413 ]]
-------------------------------------------------
ratio of variance explained by the first two principal components
 PC1         PC2 
 [0.48921491 0.16371758]
